<a href="https://colab.research.google.com/github/cds0987/Face-Verification/blob/main/Face_verification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_siamese_model(input_shape):
    base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                                   include_top=False,
                                                   weights='imagenet')
    base_model.trainable = False

    input1 = layers.Input(shape=input_shape)
    input2 = layers.Input(shape=input_shape)

    # Shared CNN base
    encoded1 = base_model(input1)
    encoded2 = base_model(input2)

    # L1 distance layer
    l1_distance = lambda x: tf.keras.backend.abs(x[0] - x[1])
    l1_layer = layers.Lambda(l1_distance)
    l1_distance_layer = l1_layer([encoded1, encoded2])

    # Flatten the output before the last Dense layer
    flatten_layer = layers.Flatten()(l1_distance_layer)

    # Fully connected layer
    fc1 = layers.Dense(128, activation='relu')(flatten_layer)
    output = layers.Dense(1, activation='sigmoid')(fc1)

    siamese_model = models.Model(inputs=[input1, input2], outputs=output)
    siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return siamese_model



In [2]:
from sklearn.model_selection import train_test_split
from itertools import combinations
import os
import random
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [3]:
def preprocess_image(image, target_size=(224, 224)):
    if isinstance(image, str):  # If 'image' is a file path
        img = Image.open(image)
    else:  # If 'image' is already a NumPy array
        img = Image.fromarray((image * 255).astype('uint8'))

    # Resize the image
    img = img.resize(target_size)

    # Convert the image to a NumPy array
    img_array = np.array(img)

    # Expand the dimensions to create a batch (MobileNetV2 expects batches)
    img_array = np.expand_dims(img_array, axis=0)

    # Preprocess the input image using MobileNetV2's preprocess_input function
    preprocessed_img_array = preprocess_input(img_array)

    return preprocessed_img_array[0]  # Remove the batch dimension


In [4]:
def generate_pairs_and_labels(directory_path):
    classes = [d for d in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, d))]
    class_indices = {c: i for i, c in enumerate(classes)}

    pairs = []
    labels = []

    for class_name in classes:
        class_path = os.path.join(directory_path, class_name)
        images = [f for f in os.listdir(class_path) if f.endswith('.jpg')]

        # Generate positive pairs
        for i in range(len(images) - 1):
            for j in range(i + 1, len(images)):
                pairs.append(
                    (
                        os.path.join(class_path, images[i]),
                        os.path.join(class_path, images[j])
                    )
                )
                labels.append(1)  # 1 indicates a positive pair

        # Generate negative pairs
        other_classes = [c for c in classes if c != class_name]
        for i in range(len(images)):
            for _ in range(5):  # Adjust the number of negative pairs as needed
                other_class = random.choice(other_classes)
                other_class_path = os.path.join(directory_path, other_class)
                other_images = [f for f in os.listdir(other_class_path) if f.endswith('.jpg')]

                pairs.append(
                    (
                        os.path.join(class_path, images[i]),
                        os.path.join(other_class_path, random.choice(other_images))
                    )
                )
                labels.append(0)  # 0 indicates a negative pair

    return pairs, labels

# Directory containing the images
directory_path = '/content/drive/MyDrive/lfw_funneled'

# Generate pairs and labels
pairs, labels = generate_pairs_and_labels(directory_path)

# Split the data into training and testing sets
train_pairs, test_pairs, train_labels, test_labels = train_test_split(pairs, labels, test_size=0.2, random_state=42)


In [5]:
print(len(train_pairs))
print(len(train_labels))


246737
246737


In [6]:
input_shape = (250, 250, 3)


In [7]:
def load_image(image_path):
    img = load_img(image_path)
    img_array = img_to_array(img)
    return img_array


In [8]:

# Create arrays for training pairs with the specified limit
max_training_pairs = 500  # Adjust as needed

# Load and create arrays for training pairs without preprocessing
train_pairs_left = np.array([load_image(pair[0]) for pair in train_pairs[:max_training_pairs]])
train_pairs_right = np.array([load_image(pair[1]) for pair in train_pairs[:max_training_pairs]])
train_labels = np.array(train_labels[:max_training_pairs])



In [10]:
print(train_pairs_left.shape)
print(train_pairs_right.shape)
print(train_labels.shape)

(500, 250, 250, 3)
(500, 250, 250, 3)
(500,)


In [15]:
max_testing_pairs = 100
test_pairs_left = np.array([load_image(pair[0]) for pair in test_pairs[:max_testing_pairs]])
test_pairs_right = np.array([load_image(pair[1]) for pair in test_pairs[:max_testing_pairs]])
test_labels = np.array(test_labels[:max_testing_pairs])

In [16]:
print(test_pairs_left.shape)
print(test_pairs_right.shape)
print(test_labels.shape)

(100, 250, 250, 3)
(100, 250, 250, 3)
(100,)


In [18]:
print(train_labels[:6])

[1 1 1 1 1 1]


In [20]:
siamese_model = build_siamese_model(input_shape)

siamese_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 250, 250, 3)]        0         []                            
                                                                                                  
 input_3 (InputLayer)        [(None, 250, 250, 3)]        0         []                            
                                                                                                  
 mobilenetv2_1.00_224 (Func  (None, 8, 8, 1280)           2257984   ['input_2[0][0]',             
 tional)                                                             'input_3[0][0]']             
                                                                                                  
 lambda (Lambda)             (None, 8, 8, 1280)           0         ['mobilenetv2_1.00_224[0][

In [21]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('best_siamese_model_weights.h5',
                              save_best_only=True,
                              save_weights_only=True,
                              monitor='val_loss',  # Monitor validation loss
                              mode='min',  # Save the weights when validation loss is minimized
                              verbose=1)


# Train the Siamese model with the ModelCheckpoint callback
siamese_model.fit([train_pairs_left, train_pairs_right], train_labels,
                  epochs=10,
                  batch_size=32,
                  validation_data=([test_pairs_left, test_pairs_right], test_labels),
                  callbacks=[checkpoint])

Epoch 1/10
16/16 [==============================] - ETA: 0s - loss: 7.3123 - accuracy: 0.7040
Epoch 1: val_loss improved from inf to 4.72806, saving model to best_siamese_model_weights.h5
16/16 [==============================] - 93s 5s/step - loss: 7.3123 - accuracy: 0.7040 - val_loss: 4.7281 - val_accuracy: 0.7400
Epoch 2/10
16/16 [==============================] - ETA: 0s - loss: 1.2110 - accuracy: 0.8720
Epoch 2: val_loss improved from 4.72806 to 1.62417, saving model to best_siamese_model_weights.h5
16/16 [==============================] - 84s 5s/step - loss: 1.2110 - accuracy: 0.8720 - val_loss: 1.6242 - val_accuracy: 0.7700
Epoch 3/10
16/16 [==============================] - ETA: 0s - loss: 0.3861 - accuracy: 0.9400
Epoch 3: val_loss improved from 1.62417 to 1.29498, saving model to best_siamese_model_weights.h5
16/16 [==============================] - 96s 6s/step - loss: 0.3861 - accuracy: 0.9400 - val_loss: 1.2950 - val_accuracy: 0.8100
Epoch 4/10
16/16 [=======================

In [22]:
siamese_model = build_siamese_model(input_shape)

# Load the best weights saved during training
siamese_model.load_weights('/content/best_siamese_model_weights.h5')

# Now, you can use the loaded model for predictions or evaluation
# Example: Make predictions on test data
predictions = siamese_model.predict([test_pairs_left, test_pairs_right])

# Evaluate the model on test data
evaluation = siamese_model.evaluate([test_pairs_left, test_pairs_right], test_labels)

print("Test Accuracy:", evaluation[1])

4/4 [==============================] - 14s 3s/step - loss: 1.1051 - accuracy: 0.8000
Test Accuracy: 0.800000011920929
